In [59]:
import pysentiment2 as py
import pandas as pd

In [60]:
data = pd.read_excel('data.xlsx')

In [61]:
data.head()

M&A SDC Deal Number Date Announced      Target Full Name  \
0           2675324040     2015-09-16         SABMiller PLC   
1           3409687020     2019-06-09           Raytheon Co   
2           3173017020     2017-12-14  21st Century Fox Inc   
3           3586676040     2020-06-11           Unilever NV   
4           3021003020     2016-10-22       Time Warner Inc   

           Acquiror Full Name    Target Macro Industry  \
0  Anheuser-Busch Inbev SA/NV         Consumer Staples   
1    United Technologies Corp              Industrials   
2              Walt Disney Co  Media and Entertainment   
3                Unilever PLC         Consumer Staples   
4                    AT&T Inc  Media and Entertainment   

   Acquiror Macro Industry Acquiror Nation   Deal Value  \
0         Consumer Staples         Belgium  1,01,490.57   
1              Industrials   United States     86831.16   
2  Media and Entertainment   United States     84197.03   
3         Consumer Staples  United Kingdom      81056.9   
4       Telecommunications   United States     79406.46   

   Acquiror Stock Price 90 Days After Announcement  \
0                                           124.80   
1                                           134.11   
2                                           109.53   
3                                            61.66   
4                                            41.45   

   Acquiror Closing Price 4 Weeks\nPrior to Announcement  Stock Price Change  \
0                                             115.17                0.083616   
1                                             137.17               -0.022308   
2                                             103.60                0.057239   
3                                              50.24                0.227309   
4                                              41.28                0.004118   

                                                News  
0  Anheuser-Busch InBev, the owner of Budweiser a...  
1  Raytheon and United Technologies\n on Sunday a...  
2  Just a few short weeks after the world first l...  
3  THE HAGUE, June 11 (Xinhua) -- British-Dutch m...  
4  The boards of the two companies unanimously ap...

In [62]:
#drop nans
data.dropna(inplace=True)

In [63]:
print(len(data))

476


In [64]:
import nltk
#data preprocessing for text analysis
#remove all stopwords
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['News'] = data['News'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [65]:
hiv4 = py.HIV4()
#using the sentiment analysis library to get sentiment scores for news articles
data['sentiment']=data['News'].apply(lambda x: hiv4.get_score(hiv4.tokenize(x)))

In [66]:
data['sentiment'].iloc[0].keys()
#seperate columns to get just polarity scores (positive/negative)
data['postive']=data['sentiment'].apply(lambda x: x['Positive'])
data['negative']=data['sentiment'].apply(lambda x: x['Negative'])
data['polarity']=data['sentiment'].apply(lambda x: x['Polarity'])

In [67]:
data.head()
data.to_csv('data_sentiment.csv')

In [71]:
#making appropriate groups for industries
group1=data[data['Acquiror Macro Industry'].isin(['Consumer Staples','Consumer Products and Services','Retail'])]
group2=data[data['Acquiror Macro Industry'].isin(['Financial Services','Real Estate'])]
group3=data[data['Acquiror Macro Industry'].isin(['Industrials','Energy and Power','Materials'])]
group4=data[data['Acquiror Macro Industry'].isin(['High Technology','Telecommunications','Media and Entertainment'])]
group5=data[data['Acquiror Macro Industry'].isin(['Healthcare'])]
#now do an anova test to see if the stock price percentage change is common across all industries
import scipy.stats as stats
result=stats.f_oneway(group1['Stock Price Change'],group2['Stock Price Change'],group3['Stock Price Change'],group4['Stock Price Change'],group5['Stock Price Change'])
print(result)

F_onewayResult(statistic=0.2085802578767678, pvalue=0.9336370235210099)


In [72]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
#further robustness checks
all_data = pd.concat([group1['Stock Price Change'], group2['Stock Price Change'],
                      group3['Stock Price Change'], group4['Stock Price Change'],
                      group5['Stock Price Change']])

group_labels = ['Group1'] * len(group1) + ['Group2'] * len(group2) + ['Group3'] * len(group3) + ['Group4'] * len(group4) + ['Group5'] * len(group5)
tukey_results = pairwise_tukeyhsd(all_data, group_labels)
print(tukey_results.summary())

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
Group1 Group2  -0.0735 0.9902 -0.5043 0.3573  False
Group1 Group3   -0.016 0.9984 -0.1656 0.1336  False
Group1 Group4  -0.0072 0.9999 -0.1648 0.1503  False
Group1 Group5   -0.043 0.9518 -0.2065 0.1205  False
Group2 Group3   0.0575 0.9958 -0.3617 0.4767  False
Group2 Group4   0.0662 0.9929 -0.3558 0.4883  False
Group2 Group5   0.0305 0.9997 -0.3938 0.4548  False
Group3 Group4   0.0087 0.9997 -0.1134 0.1309  False
Group3 Group5   -0.027 0.9793 -0.1567 0.1027  False
Group4 Group5  -0.0358 0.9552 -0.1746 0.1031  False
---------------------------------------------------
